<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/46_local_rf_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [1]:
cd~/gee/

/home/phuntsho/gee


In [2]:
ls

blog/  datapanetoken.txt  geemap-heroku/  Grok/      NestleMaxico/
data/  geemap/            gras/           links.txt  notebooks/


In [ ]:
# !pip install geemap scikit-learn

# How to use locally trained machine learning models with GEE

This notebook illustrates how to train a random forest (or any other ensemble tree estimator) locally using scikit-learn, convert the estimator into a string representation that Earth Engine can interpret, and how to apply the machine learning model with EE. **The notebook and the geemap machine learning module ([ml.py](https://geemap.org/ml/)) were contributed by [Kel Markert](https://github.com/KMarkert). A huge thank you to him.** 

In [1]:
import ee
import geemap
import pandas as pd

from geemap import ml
from sklearn import ensemble

In [2]:
geemap.ee_initialize()

## Train a model locally using scikit-learn

In this demo, we are going to use the training data from [here](https://github.com/giswqs/geemap/blob/master/examples/data/rf_example.csv). 

In [ ]:
# read the feature table to train our RandomForest model
# data taken from ee.FeatureCollection('GOOGLE/EE/DEMOS/demo_landcover_labels')

# url = "https://raw.githubusercontent.com/giswqs/geemap/master/examples/data/rf_example.csv"
# df = pd.read_csv(url)

df = pd.read_csv('training_samples.csv')

In [ ]:
df

In [5]:
# specify the names of the features (i.e. band names) and label
# feature names used to extract out features and define what bands

feature_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'BSI','NDVI']
label = "class"

In [ ]:
# get the features and labels into seperate variables
X = df[feature_names]
y = df[label]

In [ ]:
# create a classifier and fit
n_trees = 100
rf = ensemble.RandomForestClassifier(n_trees).fit(X,y)

## Convert a sklearn classifier object to a list of strings

In [ ]:
# convert the estimator into a list of strings
# this function also works with the ensemble.ExtraTrees estimator
trees =  ml.rf_to_strings(rf,feature_names)

In [ ]:
# print the first tree to see the result
print(trees[0])

In [ ]:
print(trees[1])

In [ ]:
# number of trees we converted should equal the number of trees we defined for the model
len(trees) == n_trees

## Convert sklearn classifier to GEE classifier

At this point you can take the list of strings and save them locally to avoid training again. However, we want to use the model with EE so we need to create an ee.Classifier and persist the data on ee for best results.

In [ ]:
# create a ee classifier to use with ee objects from the trees
ee_classifier = ml.strings_to_classifier(trees)

In [ ]:
ee_classifier.getInfo()

## Classify image using GEE classifier

In [3]:
# Make a cloud-free Landsat 5 TOA composite (from raw imagery).
l5 = ee.ImageCollection('LANDSAT/LT05/C01/T1');
composite = ee.Algorithms.Landsat.simpleComposite(
  collection= l5.filterDate('2000-01-01', '2000-12-31')
)

In [4]:
#Create BSII and composite
ndvi = composite.normalizedDifference(['B4', 'B3']).rename('NDVI')

# Add NDVI image as Band
comp = ee.Image.cat(composite,ndvi)

bsi = composite.expression(
    '((Red+SWIR) - (NIR+Blue)) / ((Red+SWIR) + (NIR+Blue))', {
      'NIR': composite.select('B4'),
      'Red': composite.select('B3'),
      'Blue': composite.select('B1'),
      'SWIR': composite.select('B5')
}).rename('BSI')

# Add BSI image as Band
image = ee.Image.cat(comp,bsi)

In [ ]:
# classify the image using the classifier we created from the local training
# note: here we select the feature_names from the image that way the classifier knows which bands to use
classified = image.select(feature_names).classify(ee_classifier)

In [7]:
# display results
Map = geemap.Map()
Map.setCenter(-96.6142, 17.4856,9)
Map.addLayer(image,{"bands": ['B7', 'B5', 'B3'], "min":15, "max": 50, "gamma":1}, 'image')
# Map.addLayer(classified, {"min": 1, "max": 5, "palette": ['green','purple','orange', 'red', 'blue']},'classification')

Map

Map(center=[17.4856, -96.6142], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

### Import shapefile

In [16]:
Map = geemap.Map() #center=(-95.6142, 17.4856), zoom=11
Map.setCenter(-95.6142, 17.4856,6)
# countries_shp = '../data/countries.shp'
maxico_states = './NestleMaxico/Nestle_Maxico_Outline_simpl.shp'
states = geemap.shp_to_ee(maxico_states)

Map.addLayer(states, {}, 'States')
Map.addLayer(states.geometry().bounds(), {}, 'States1')
Map

Map(center=[17.4856, -95.6142], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [99]:
pwd

'/home/phuntsho/gee'

In [ ]:
geemap.ee_to_shp(ee.FeatureCollection(roi), filename='../NestleMaxico/roi.shp')

In [ ]:
ls

Yay!! 🎉 Looks like our example works. Don't party too much because there is a catch...

This workflow has several limitations particularly due to how much data you can pass from the client to the server and how large of a model ee can acutally handle. EE can only handle 40MB of data passed to the server, so if you have a lot of large decision tree strings then this will not work. Also, creating a classifier from strings has limitation (see this ee-forum discussion: https://groups.google.com/g/google-earth-engine-developers/c/lFFU1GBPzi8/m/6MewQk1FBwAJ), this is again limited by string lengths when ee creates a computation graph.

So, you can use this but know you will probably run into errors when training large models.

## Save trees to the cloud and call the model to apply again

Now we have the strings in a format that ee can use, we want to save it for later use. There is a function to export a list of tree strings to a feature collection. The feature collection will have a pro

In [9]:
user_id = geemap.ee_user_id()
user_id

'users/phuntshophuntsho'

In [10]:
# specify asset id where to save trees
# be sure to change <user_name> to your ee user name
asset_id = user_id +  "/random_forest_strings_l5_index"
asset_id

'users/phuntshophuntsho/random_forest_strings_l5_index'

In [ ]:
# kick off an export process so it will be saved to the ee asset
# ml.export_trees_to_fc(trees,asset_id)

# this will kick off an export task, so wait a few minutes before moving on

In [11]:
# read the exported tree feature collection
rf_fc = ee.FeatureCollection(asset_id)

# convert it to a classifier, very similar to the `ml.trees_to_classifier` function
another_classifier = ml.fc_to_classifier(rf_fc)

# classify the image again but with the classifier from the persisted trees
classified = image.select(feature_names).classify(another_classifier)

In [12]:
# display results
# we should get the exact same results as before
Map = geemap.Map() #center=(-95.6142, 17.4856), zoom=11
Map.setCenter(-95.6142, 17.4856,6)

Map.addLayer(image,{"bands": ['B7', 'B5', 'B3'], "min":15, "max": 50, "gamma":1}, 'image')
Map.addLayer(classified, {"min": 0, "max": 4, "palette": ['green', 'purple', 'orange', 'red', 'blue']},'classification')

Map

Map(center=[17.4856, -95.6142], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [14]:
# Export to asset
task = ee.batch.Export.image.toAsset(**{
    'image':classified.uint8(),
    'description':'Classified2000_all',
    'assetId': user_id + '/NestleMaxico/Classified2000_all',
    'region': states.geometry().bounds(),
    'scale': 30,
    'maxPixels': 1e13
})
task.start()

## Save trees locally

In [ ]:
import os
out_csv = os.path.expanduser("~/Downloads/trees.csv")

In [ ]:
ml.trees_to_csv(trees, out_csv)

In [ ]:
another_classifier = ml.csv_to_classifier(out_csv)

In [ ]:
classified = image.select(feature_names).classify(another_classifier)

In [ ]:
# display results
# we should get the exact same results as before
Map = geemap.Map(center=(37.75,-122.25), zoom=11)

Map.addLayer(image,{"bands": ['B7', 'B5', 'B3'], "min":0.05, "max": 0.55, "gamma":1.5}, 'image')
Map.addLayer(classified, {"min": 0, "max": 2, "palette": ['red', 'green', 'blue']},'classification')

Map